In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile("/content/drive/MyDrive/transformed_oct.zip", 'r')
zip_ref.extractall("/content/")
zip_ref.close()

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten, Dropout, BatchNormalization, ReLU
from tensorflow.keras.models import Model
from tensorflow.keras.applications import InceptionV3, ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy, Precision, Recall, AUC
from tensorflow.keras.callbacks import ReduceLROnPlateau, Callback
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

In [ ]:
pretext_path = '/content/Pretext Learning'
pretext_datagen = ImageDataGenerator(rescale=1./255)
pretext_generator = pretext_datagen.flow_from_directory(
    pretext_path,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical'
)

Found 65279 images belonging to 4 classes.


In [ ]:
base_model = ResNet50(weights=None, include_top=False, input_shape=(224, 224, 3))

# Pretext task
#x = base_model.get_layer("conv3_block4_out").output
x = base_model.layers[-1].output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
output = Dense(4, activation='softmax')(x)

# Model
pretext_model = Model(inputs=base_model.input, outputs=output)

# Freeze layers
for layer in base_model.layers[-10:]:  # Unfreeze the last 10 layers, for example
    layer.trainable = True

# Compile
pretext_model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=0.005), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train on pretext
pretext_model.fit(pretext_generator, epochs=10)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1020/1020 ━━━━━━━━━━━━━━━━━━━━ 664s 581ms/step - accuracy: 0.6495 - loss: 0.9046
Epoch 2/10
1020/1020 ━━━━━━━━━━━━━━━━━━━━ 565s 552ms/step - accuracy: 0.9132 - loss: 0.1947
Epoch 3/10
1020/1020 ━━━━━━━━━━━━━━━━━━━━ 614s 545ms/step - accuracy: 0.9257 - loss: 0.1642
Epoch 4/10
1020/1020 ━━━━━━━━━━━━━━━━━━━━ 562s 549ms/step - accuracy: 0.9284 - loss: 0.1826
Epoch 5/10
1020/1020 ━━━━━━━━━━━━━━━━━━━━ 561s 548ms/step - accuracy: 0.9391 - loss: 0.1467
Epoch 6/10
1020/1020 ━━━━━━━━━━━━━━━━━━━━ 561s 549ms/step - accuracy: 0.9569 - loss: 0.0974
Epoch 7/10
1020/1020 ━━━━━━━━━━━━━━━━━━━━ 562s 548ms/step - accuracy: 0.9612 - loss: 0.0909
Epoch 8/10
1020/1020 ━━━━━━━━━━━━━━━━━━━━ 561s 547ms/step - accuracy: 0.9648 - loss: 0.0858
Epoch 9/10
1020/1020 ━━━━━━━━━━━━━━━━━━━━ 561s 547ms/step - accuracy: 0.9678 - loss: 0.0750
Epoch 10/10
1020/1020 ━━━━━━━━━━━━━━━━━━━━ 563s 547ms/step - accuracy: 0.9687 - loss: 0.0727


In [ ]:
weights_path = '/content/drive/MyDrive/weights/Full_Model/pretext_task_weights.weights.h5'
pretext_model.save_weights(weights_path)

print(f"Weights saved at: {weights_path}")

Weights saved at: /content/drive/MyDrive/weights/Full_Model/pretext_task_weights.weights.h5


In [ ]:
pretext_model.load_weights('/content/drive/MyDrive/weights/Full_Model/pretext_task_weights.weights.h5',skip_mismatch=True)

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 434 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
# Terminate
import os, signal
os.kill(os.getpid(), signal.SIGKILL)